In [7]:
%load_ext autoreload
%autoreload 2

import rospy
from rospy_tutorials.srv import AddTwoInts, AddTwoIntsResponse
import jupyros
from turtlesim.srv import Spawn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
rospy.init_node("adder")

In [3]:
def handle_add_two_ints(req):
    print("Returning [%s + %s = %s]"%(req.a, req.b, (req.a + req.b)))
    return AddTwoIntsResponse(req.a + req.b)

In [4]:
%%thread_cell

srv = rospy.Service('add_two_ints', AddTwoInts, handle_add_two_ints)

Output(layout=Layout(border='1px solid gray'))

In [5]:
jupyros.client('add_two_ints', AddTwoInts)

Returning [3 + 5 = 8]


In [ ]:
# !rosrun turtlesim turtlesim_node

In [9]:
jupyros.client('spawn', Spawn)

Service call failed: service [/spawn] responded with an error: b''
Service call failed: service [/spawn] responded with an error: b''
Service call failed: service [/spawn] responded with an error: b''
Service call failed: service [/spawn] responded with an error: b''
Service call failed: service [/spawn] responded with an error: b''
